In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import os

# Path ke file CSV di Google Drive
cl_file_path = 'realbanget_CL.csv'
cd_file_path = 'realbanget_CD.csv'

# Fungsi untuk menyimpan hasil ke file CSV berdasarkan nilai alpha_root
def save_hasil_to_csv(df_results, alpha_real):
    # Nama file akan berdasarkan nilai alpha_root
    filename = f'hasil_alpha_cenderung_{alpha_real}.csv'
    
    # Cek apakah file dengan nama tersebut sudah ada
    if os.path.exists(filename):
        # Jika sudah ada, tambahkan data baru ke file yang sudah ada
        df_results.to_csv(filename, mode='a', header=False, index=False)
    else:
        # Jika belum ada, buat file baru dan simpan data dengan header
        df_results.to_csv(filename, mode='w', header=True, index=False)
    
    print(f'Hasil disimpan di {filename}')

# Fungsi untuk menyimpan hasil ke file CSV dengan nama yang selalu berbeda jika ditemukan nama file yang sama
def save_results_to_csv(df_results, base_filename='root_final.csv'):
    # Cek apakah file dengan nama tersebut sudah ada
    filename = base_filename
    file_counter = 1
    
    while os.path.exists(filename):
        # Jika sudah ada, tambahkan nomor di belakangnya
        filename = f'{base_filename.split(".")[0]}{file_counter}.csv'
        file_counter += 1

    # Simpan file dengan nama baru
    df_results.to_csv(filename, index=False)
    print(f'Hasil disimpan di {filename}')

# Fungsi untuk membaca data polar dari dua file CSV dan melakukan interpolasi
def interpolate_polar(cl_file_path, cd_file_path, Re_target, alpha_target):
    # Baca data dari file CSV
    cl_data = pd.read_csv(cl_file_path)
    cd_data = pd.read_csv(cd_file_path)

    # Temukan dua nilai Re terdekat untuk Cl
    Re_values_cl = cl_data['Re'].unique()
    Re_values_sorted_cl = np.sort(Re_values_cl)

    if Re_target > Re_values_sorted_cl.max():
        Re_below_cl = Re_values_sorted_cl.max()
        Re_above_cl = Re_values_sorted_cl.max()
    elif Re_target < Re_values_sorted_cl.min():
        Re_below_cl = Re_values_sorted_cl.min()
        Re_above_cl = Re_values_sorted_cl.min()
    else:
        Re_below_cl = Re_values_sorted_cl[Re_values_sorted_cl <= Re_target].max() if len(Re_values_sorted_cl[Re_values_sorted_cl <= Re_target]) > 0 else None
        Re_above_cl = Re_values_sorted_cl[Re_values_sorted_cl >= Re_target].min() if len(Re_values_sorted_cl[Re_values_sorted_cl >= Re_target]) > 0 else None

    if Re_below_cl is None or Re_above_cl is None:
        raise ValueError(f"Tidak ada data untuk Re di sekitar {Re_target} pada file CL")

    # Temukan dua nilai Re terdekat untuk Cd
    Re_values_cd = cd_data['Re'].unique()
    Re_values_sorted_cd = np.sort(Re_values_cd)

    if Re_target > Re_values_sorted_cd.max():
        Re_below_cd = Re_values_sorted_cd.max()
        Re_above_cd = Re_values_sorted_cd.max()
    elif Re_target < Re_values_sorted_cd.min():
        Re_below_cd = Re_values_sorted_cd.min()
        Re_above_cd = Re_values_sorted_cd.min()
    else:
        Re_below_cd = Re_values_sorted_cd[Re_values_sorted_cd <= Re_target].max() if len(Re_values_sorted_cd[Re_values_sorted_cd <= Re_target]) > 0 else None
        Re_above_cd = Re_values_sorted_cd[Re_values_sorted_cd >= Re_target].min() if len(Re_values_sorted_cd[Re_values_sorted_cd >= Re_target]) > 0 else None

    if Re_below_cd is None or Re_above_cd is None:
        raise ValueError(f"Tidak ada data untuk Re di sekitar {Re_target} pada file CD")

    # Filter data berdasarkan nilai Re yang ditemukan untuk Cl
    data_below_cl = cl_data[cl_data['Re'] == Re_below_cl]
    data_above_cl = cl_data[cl_data['Re'] == Re_above_cl]

    # Filter data berdasarkan nilai Re yang ditemukan untuk Cd
    data_below_cd = cd_data[cd_data['Re'] == Re_below_cd]
    data_above_cd = cd_data[cd_data['Re'] == Re_above_cd]

    # Ambil data alpha dan CL untuk kedua nilai Re pada file Cl
    alpha_data_below_cl = data_below_cl['Alpha'].values
    CL_data_below = data_below_cl['CL'].values

    alpha_data_above_cl = data_above_cl['Alpha'].values
    CL_data_above = data_above_cl['CL'].values

    # Ambil data alpha dan CD untuk kedua nilai Re pada file Cd
    alpha_data_below_cd = data_below_cd['Alpha'].values
    CD_data_below = data_below_cd['CD'].values

    alpha_data_above_cd = data_above_cd['Alpha'].values
    CD_data_above = data_above_cd['CD'].values

    # Interpolasi nilai CL berdasarkan alpha untuk kedua nilai Re pada file Cl
    CL_interp_func_below = interp1d(alpha_data_below_cl, CL_data_below, fill_value="extrapolate")
    CL_interp_func_above = interp1d(alpha_data_above_cl, CL_data_above, fill_value="extrapolate")

    CL_below = CL_interp_func_below(alpha_target)
    CL_above = CL_interp_func_above(alpha_target)

    # Interpolasi nilai CD berdasarkan alpha untuk kedua nilai Re pada file Cd
    CD_interp_func_below = interp1d(alpha_data_below_cd, CD_data_below, fill_value="extrapolate")
    CD_interp_func_above = interp1d(alpha_data_above_cd, CD_data_above, fill_value="extrapolate")

    CD_below = CD_interp_func_below(alpha_target)
    CD_above = CD_interp_func_above(alpha_target)

    # Interpolasi linier antara kedua nilai Re untuk mendapatkan CL dan CD
    CL_interp = np.interp(Re_target, [Re_below_cl, Re_above_cl], [CL_below, CL_above])
    CD_interp = np.interp(Re_target, [Re_below_cd, Re_above_cd], [CD_below, CD_above])

    return CL_interp, CD_interp

# Parameter Desain Awal
num_blades = 8
num_elements = 21
rho = 1.225            # Kerapatan udara (kg/m^3)
root_length = 0.725    # Panjang dari root ke pusat rotor (m)
tip_length = 1.45      # Panjang dari tip ke pusat rotor (m)
viscosity = 1.81 * 10**-5 # Dynamic Viscosity

# Inisialisasi parameter untuk perhitungan Const
D_f = tip_length * 2  # Menggunakan tip_length untuk menghitung diameter fan
D_h = root_length * 2  # Menggunakan root_length untuk menghitung diameter hub
RPM = 900  # Menggunakan nilai RPM yang sesuai dengan data

# Menghitung kecepatan aksial (V_f)
# V_f = 143    #Dihitung momentum theory dengan dilakukan rata2 station2 - 3
Vol = 272.3
V_f = Vol / ((np.pi / 4) * (D_f**2 - D_h**2))

# Menghitung RPS
RPS = RPM / 60
omega =  2 * np.pi * RPS
Const1 = (np.pi * RPS * D_f) / 1000

# Inisialisasi daya yang diinginkan dan perhitungan daya awal
P_des = 300000    # Daya yang diinginkan (Watt)
P_A = 0             # Daya yang tersedia (Watt)
iterations = 0      # Menghitung iterasi

# Variasi sudut serang untuk setiap sudu dan elemen
alpha_variation = [0, 2, 4.15]
max_iterations = 1000  # Batas maksimum iterasi untuk keamanan
initial_Const = Const1  # Inisialisasi nilai Const
i = 0

# Fungsi untuk menghitung parameter elemen dan power available
def compute_element_parameters(j, V_f, omega, root_length, tip_length, num_elements, rho, viscosity, Const, previous_chord_length):
    r = root_length + (tip_length - root_length) * j / (num_elements - 1)
    dr = (tip_length - root_length) / (num_elements)

    V = Const / r
    U = 2 * np.pi * r * RPS

    W = np.sqrt(V_f**2 + (U - (V / 2))**2)

    phi = np.arctan(V_f / (U - (V / 2)))
    c_cq = (4 * np.pi * r * V_f * V) / (num_blades * W**2)

    delta_cq = 1
    Cq1 = 0
    alpha_target = alpha_real

    while delta_cq > 0.001:
        Cq1 += 0.001
        chord_length = c_cq / Cq1

        Re = (rho * W * chord_length) / viscosity

        CL, CD = interpolate_polar(cl_file_path, cd_file_path, Re, alpha_target)

        # Menghitung Ct berdasarkan CL dan CD
        Ct = CL * np.cos(phi) - CD * np.sin(phi)
        Cq2 = CL * np.sin(phi) + CD * np.cos(phi)

        delta_cq = Cq2 - Cq1

        if abs(delta_cq) > 0.001:
            continue

    Rey = (rho * W * chord_length) / viscosity

    C_R = c_cq / (Cq2 * tip_length)

    # Menghitung delta p_23 berdasarkan Ct
    delta_p23 = (num_blades * chord_length / (4 * np.pi * r)) * rho * W**2 * Ct

    # Menghitung power untuk elemen ini
    power_available = delta_p23 * 2 * np.pi * r * V_f * dr

    # Menghitung thrust
    thrust_momentum = delta_p23 * 2 * np.pi * r * dr
    thrust_blade_elements = num_blades * rho * W**2 * chord_length * Ct * dr / 2

    # Menghitung torque
    torque_momentum = rho * V_f * 2 * np.pi * r**2 * V * dr
    torque_blade_elements = num_blades * rho * W**2 * chord_length * Cq2 * r * dr / 2

    # Menghitung daya yang diperlukan
    power_required = rho * V_f * 2 * np.pi * r**2 * V * omega * dr

    return {
        'X': r / tip_length,
        'C/R': C_R,
        'BETA': np.degrees(phi),
        'ALPHA': alpha_target,
        'REYN': Re,
        'RAD(MM)': r * 1000,
        'CRD(MM)': chord_length * 1000,
        'Power Available': power_available,
        'Power Required': power_required,
        'Total Thrust Momentum': thrust_momentum,
        'Total Thrust Blade Elements': thrust_blade_elements,
        'Total Torque Momentum': torque_momentum,
        'Total Torque Blade Elements': torque_blade_elements,
        'V Swirl': V,
        'CL': CL,
        'CD': CD,
        'Const' : Const
    }

while i < len(alpha_variation):
    alpha_real = alpha_variation[i]
    iterations = 0  # Reset jumlah iterasi untuk setiap alpha_variation
    Const = initial_Const  # Reset nilai Const
    P_A = 0  # Reset daya tersedia
    P_R = 0  # Reset daya yang diperlukan

    print(f"Memulai iterasi untuk alpha_variation[{i}] = {alpha_real}")
    
    while abs(P_des - P_A) > 0.01 and iterations < max_iterations:
        iterations += 1
        results = []
        total_power = 0
        total_power_required = 0
        total_thrust_momentum = 0
        total_thrust_blade_elements = 0
        total_torque_momentum = 0
        total_torque_blade_elements = 0

        # Loop untuk setiap elemen
        for j in range(num_elements):
            element_result = compute_element_parameters(
                j, V_f, omega, root_length, tip_length, num_elements, rho, viscosity, Const, initial_Const
            )
            results.append(element_result)
            total_thrust_momentum += element_result['Total Thrust Momentum']
            total_thrust_blade_elements += element_result['Total Thrust Blade Elements']
            total_torque_momentum += element_result['Total Torque Momentum']
            total_torque_blade_elements += element_result['Total Torque Blade Elements']
            total_power += element_result['Power Available']
            total_power_required += element_result['Power Required']

        P_A = total_power
        P_R = total_power_required

        if abs(P_A - P_des) > 0.01:
            Const = (P_des / P_A) * Const
            print(f"Iterasi {iterations}: P_A = {P_A}, P_R = {P_R}, Const = {Const}")
            continue

        print(f"Berhasil konvergen: P_A = {P_A}, iterasi = {iterations}")
        break

    df_hasil = pd.DataFrame({
        'alpha_root': [alpha_real],
        'alpha_tip': [alpha_real],
        'P_A': [P_A],
        'P_R': [P_R],
        'total_thrust_momentum': [total_thrust_momentum],
        'total_thrust_blade_elements': [total_thrust_blade_elements],
        'total_torque_momentum': [total_torque_momentum],
        'total_torque_blade_elements': [total_torque_blade_elements],
        'efisiensi': [(P_A / P_R) * 100 if P_R != 0 else 0]
    })

    df_results = pd.DataFrame(results)

    save_results_to_csv(df_results)
    save_hasil_to_csv(df_hasil, alpha_real)

    print(f"Iterasi selesai untuk alpha_real = {alpha_real}")
    print(f"Total iterasi: {iterations}")
    print("-" * 50)

    if iterations >= max_iterations:
        print(f"Iterasi maksimum tercapai untuk alpha_real = {alpha_real}. Tidak konvergen.")

    i += 1


Memulai iterasi untuk alpha_variation[0] = 0
Iterasi 1: P_A = 672.62494914813, P_R = 4296.293942208789, Const = 60.95192303120768
Iterasi 2: P_A = 1323869.7661784696, P_R = 1916206.326118285, Const = 13.812217316621794
Iterasi 3: P_A = 391715.4712732487, P_R = 434228.43584902067, Const = 10.578252580930213
Iterasi 4: P_A = 303426.36916757113, P_R = 332559.0646988637, Const = 10.458800212339064
Iterasi 5: P_A = 300169.53395663144, P_R = 328803.72158611275, Const = 10.452893144562253
Iterasi 6: P_A = 299904.85989724717, P_R = 328618.01521164866, Const = 10.456209160608738
Iterasi 7: P_A = 300053.4249007092, P_R = 328722.26411159785, Const = 10.454347419032601
Iterasi 8: P_A = 299970.01041455154, P_R = 328663.73468695657, Const = 10.455392595331384
Iterasi 9: P_A = 300016.8377208564, P_R = 328696.5929354914, Const = 10.4548058116585
Iterasi 10: P_A = 299990.5474755679, P_R = 328678.1456325989, Const = 10.455135236396044
Iterasi 11: P_A = 300005.3068822355, P_R = 328688.50208625395, Const 